This notebook develops Suits index (Measure of tax (here toll!) progressivity) at city level. Suits index is calculated by measuring the area between lorenz curve and proportional line (45 degree line.) Value is between -1 (regressive-fare burden is on low-income individuals) and 1 (progressive-fare burden is on high-income individuals).
Preprocessing step (sql):
Need to run sql code "Suits_index_city.sql" to have a table with trip data and the corresponding cities' name (TOT_W_NULL_C)
Here we develop a mixed effect model (Dependant variable is fair paid during one year($) and independent variables include (fixed effect: cities, hh income, and interaction between cities and hh income, random effect: tracts.) Variables are log-scaled
We simulate the hhs'income from the distribution of cities and plug into the model to calculate the fair share for the hh and then use income and fair to plot the lorenz curve.
Here we only did analysis on the 9 city with the highest number of trips

In [ ]:
from pysqlcipher3 import dbapi2 as sqlite
from os import environ
import pandas as pd
import numpy as np

In [ ]:
keynow = environ['HOT_KEY']
conn = sqlite.connect('hot_2.1.db')
conn.execute('pragma key=\"x\''+keynow+'\'\"')

In [ ]:
# Creating a dataframe that includes fips, corresponding city, their share of toll, number of accounts, and number of their trips from that fips
dffips = pd.read_sql_query("SELECT fips, tract, city_new_final, SUM(toll),count(Distinct(id)),
                           count(Distinct(trip_id)) FROM TOT_W_NULL_C GROUP BY fips", conn)
dffips.rename(columns = {'city_new_final':'new_city'},inplace = True)
dffips['fareshare'] = dffips['SUM(toll)']/dffips['count(Distinct(id))']

In [ ]:
# Reading census block group data calculate mean income 
dfcensus = pd.read_csv('block_group_census_estimates_wide_original_bins.csv')
dfcensus['mean_income_hh'] = (dfcensus['pc_income']*dfcensus['population'])/dfcensus['households']
dfcensus.rename(columns = {'fips_code':'fips'},inplace = True)

In [ ]:
# Merging the the fips table and census block group table 
dfcensus['fips'] = dfcensus['fips'].astype(str)
dffips['fips'] = dffips['fips'].astype(str)
dfmergetemp = pd.merge(dffips,dfcensus,on='fips')
dfmergetemp['log_scaled_mean_income_hh'] = np.log(dfmergetemp['mean_income_hh'])
dfmergetemp['log_fareshare'] = np.log(dfmergetemp['fareshare'])
dfmerge = dfmergetemp[['fips','tract_x','new_city','SUM(toll)','count(Distinct(id))','count(Distinct(trip_id))','fareshare','county_name','mean_income_hh','log_scaled_mean_income_hh','log_fareshare']]
dfmerge.rename(columns={'tract_x':'tract','new_city':'city','SUM(toll)':'total_toll','count(Distinct(id))':'total_accounts',
                        'count(Distinct(trip_id))':'total_trips','fareshare':'fare_per_account','mean_income_hh':'mean_hh_income',
                        'log_scaled_mean_income_hh':'log_scaled_mean_hh_income','log_fareshare':'log_scaled_fare_per_account'},inplace = True)
# Removing fips code without city
dfmerge_w_na = dfmerge.dropna()

In [ ]:
#### Looking at cities with highest number of trips
dfmerge_w_na = dfmerge_w_na[dfmerge_w_na['city'].isin(['BOTHELL','KIRKLAND','EVERETT','LYNNWOOD','WOODINVILLE','SEATTLE','BELLEVUE','SNOHOMISH','REDMOND'])]
dfmerge_w_na.sort_values(by = ['total_trips'],ascending = False, inplace = True)
dfmerge_w_na['tract_name'] = 'tract' + dfmerge_w_na['tract'].astype(str)

In [ ]:
### Plotting
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette(sns.color_palette("hls", 9))
fig, ax = plt.subplots(figsize=(30, 10))
sns.lmplot(x = 'log_scaled_mean_hh_income',y='log_scaled_fare_per_account',hue='city',data= dfmerge_w_na, height = 10)
plt.savefig('Figures/incomevfare')


In [ ]:
# Modelling
import statsmodels.api as sm
import statsmodels.formula.api as smf
model = smf.mixedlm(" log_scaled_fare_per_account ~ log_scaled_mean_hh_income*city ", dfmerge_w_na, groups= "tract_name")
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# This code is inspired by Cory's code on fitting a distribution to a histogram
from sympy import Symbol
from scipy.optimize import minimize 
from scipy.stats import burr
from scipy import stats

bins = np.array([0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 75, 100, 125, 150, 200, np.inf])
n_bins = len(bins) - 1
bins_l = bins[0:n_bins]
bins_r = bins[1:(n_bins + 1)]

def burr_fcn(params,counts_in,bins_r_in,bins_left_in):
    estimated_percentage = (burr.cdf(bins_r_in, c= params[0], d = params[1], loc=0, scale=params[2]) -  burr.cdf(bins_left_in, c= params[0], d = params[1], loc=0, scale=params[2]))
    estimated_percentage = np.where(estimated_percentage <= 1e-99 ,1e-99, estimated_percentage)
    log_lik = -np.sum(counts_in*100*np.log(estimated_percentage))
    return(log_lik)

def opt_fcn(fcn_in,bins_l_in,counts_in, city_in):
    bnds = ((0, None), (0, None), (0, None))
    # Needed to change initial start point for WOODINVILLE
    if(city_in == 'WOODINVILLE'):
        return minimize(fcn_in, np.array([2,1,120]),args = (counts_in,bins_r,bins_l),bounds=bnds) # method = 'BFGS
    else:
        return minimize(fcn_in, np.array([4,0.6,120]),args = (counts_in,bins_r,bins_l),bounds=bnds) # method = 'BFGS'

In [ ]:
def plot_lorenz_curve(fare_share_in,income_share_in,num_share_in,axes_in,ind_i_in,ind_j_in,name_in):
    cum_fare_share = np.cumsum(fare_share_in)
    cum_num_share = np.cumsum(num_share_in)
    cum_income_share = np.cumsum(income_share_in)
    l1 = axes_in[ind_i_in,ind_j_in].plot(cum_income_share*100,cum_fare_share*100,color = 'red', label = 'Lorenz curve (Toll burden)')
    l2 = axes_in[ind_i_in,ind_j_in].plot(cum_income_share*100,cum_num_share*100,color = 'black',  label = 'Lorenz curve (Income inequality)')
    l3 = axes_in[ind_i_in,ind_j_in].plot(np.arange(0,1.1,0.1)*100,np.arange(0,1.1,0.1)*100,color = 'blue',  label = 'Perfect Equality')
    axes_in[ind_i_in,ind_j_in].set_title(str(name_in))
    if (name_in =='REDMOND'):
        legend.extend([l1,l2,l3])

In [ ]:
def cal_suits_index(fare_share_in,income_share_in):
    cum_fare_share = np.cumsum(fare_share_in)
    cum_income_share = np.cumsum(income_share_in)
    suits_index = 0
    for i in range(1,cum_fare_share.shape[0]):
        suits_index = suits_index + 0.5*(cum_fare_share[i]+cum_fare_share[i-1])*(cum_income_share[i]-cum_income_share[i-1])
    return 2*(1/2-suits_index)

In [ ]:
bins0 = np.array([0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 75, 100, 125, 150, 200])
bins1 = np.array([10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 75, 100, 125, 150, 200, 1000])
bins2 = np.array([10, 5, 5, 5, 5, 5, 5, 5, 5, 10, 15, 25, 25, 25, 50, 800])
legend =[]
dfcensus = pd.read_csv('block_group_census_estimates_wide_original_bins.csv')
n1 = 3
n2 = 3
fig, axes = plt.subplots(n1,n2,figsize=(12,12),sharex=True,sharey=True)
fig1, ax1 = plt.subplots(n1,n2,figsize=(12,12),sharex=True,sharey=True)
indi = 0
indj = 0
for i in ['BELLEVUE','BOTHELL','KIRKLAND','EVERETT','LYNNWOOD','WOODINVILLE','SEATTLE','SNOHOMISH','REDMOND']:
    dffips =  dfmerge_w_na[ dfmerge_w_na['city'] == i].fips
    dfnum =  dfmerge_w_na[ dfmerge_w_na['city'] == i][['fips','total_accounts']]
    dfcensus_city = dfcensus[dfcensus['fips_code'].isin(dffips)]
    dfcensus_income = dfcensus_city[['fips_code','inc_000_010k', 'inc_010_015k', 'inc_015_020k',
       'inc_020_025k', 'inc_025_030k', 'inc_030_035k', 'inc_035_040k',
       'inc_040_045k', 'inc_045_050k', 'inc_050_060k', 'inc_060_075k',
       'inc_075_100k', 'inc_100_125k', 'inc_125_150k', 'inc_150_200k',
       'inc_200_infk']]
    dfnum.set_index('fips', inplace = True)
    dfnum.sort_index(inplace = True)
    dfcensus_income.set_index('fips_code', inplace = True)
    dfcensus_income.sort_index(inplace = True)
    dfproduct = dfcensus_income[['inc_000_010k', 'inc_010_015k', 'inc_015_020k',
       'inc_020_025k', 'inc_025_030k', 'inc_030_035k', 'inc_035_040k',
       'inc_040_045k', 'inc_045_050k', 'inc_050_060k', 'inc_060_075k',
       'inc_075_100k', 'inc_100_125k', 'inc_125_150k', 'inc_150_200k',
       'inc_200_infk']].mul(dfnum["total_accounts"].values,axis='index')
    dfcensusincome = dfproduct.sum()/dfnum['total_accounts'].sum()
    counts = dfcensusincome
    count = np.sum(counts)
    # Finding the optimal parameters for the distribution 
    result = opt_fcn(burr_fcn,bins_l,counts,i)
    if result.success:
        newbins = np.array([0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 75, 100, 125, 150, 200,1000])
        x1 = np.linspace(0,1000,1000)
        y1 = burr.pdf(x1, c = result.x[0] , d=result.x[1] , scale = result.x[2])
        ax1[indi,indj].bar(bins0, np.array(dfcensusincome/bins2), width = bins2, ec="r", align="edge", alpha = 0.2)
        ax1[indi,indj].plot(x1,y1)
        ax1[indi,indj].set_title(str(i))
        simnum = 10000000
        r = burr.rvs(c = result.x[0] , d=result.x[1] , scale = result.x[2], size=simnum)
        # Sorting income
        income_sim_sort =  np.sort(r*1000)
        # Calcualting the log
        income_sim = np.log(income_sim_sort)
        # Fixing the paramters needed for applying the fitted mixed model
        name = 'T.' + str(i)
        city_coeff = 'city[' + name  + ']'
        income_coeff = 'log_scaled_mean_hh_income:city[' + name + ']'
        if(i == 'BELLEVUE'):
            fare = np.exp(mdf.params.get('Intercept') + mdf.params.get('log_scaled_mean_hh_income')* income_sim)
        else:
            fare = np.exp(mdf.params.get('Intercept') + mdf.params.get(city_coeff) + mdf.params.get(income_coeff)* income_sim +
                          mdf.params.get('log_scaled_mean_hh_income')* income_sim)
        final = pd.DataFrame({'fare_share':fare ,'income_share':income_sim_sort,'num_share':np.repeat(1,simnum)})
        final['fare_share_norm']= final['fare_share']/final['fare_share'].sum()
        final['income_share_norm']= final['income_share']/final['income_share'].sum()
        final['num_share_norm'] = final['num_share']/final['num_share'].sum()
        plot_lorenz_curve(final['fare_share_norm'],final['income_share_norm'],final['num_share_norm'],axes,indi,indj,i)
        if(indi < n1-1):
            indi = indi + 1
        else:
            indi = 0
            indj = indj + 1
        print("done")
    else:
        raise ValueError(result.message)
fig.text(0.06, 0.5, 'Accumulated % of total fare or Accumulated % of total accounts', va='center', rotation='vertical')
fig.text(0.5, 0.06, 'Accumulated % of total income', va='center', rotation='horizontal')
axes[0,2].legend(legend,['Lorenz curve (Toll burden)','Lorenz curve (Income inequality)','Perfect Equality'],loc="upper right")
plotname1 = 'Figures/suits_index'
plotname2 = 'Figures/distribution'
fig.savefig(plotname1) 
fig1.savefig(plotname2)